In [1]:
%%bash
tree data

data
├── UKBB
│   ├── processed
│   │   ├── DeepSEA
│   │   │   └── variants_no_indels.chr12.tsv
│   │   ├── I10.gwas.imputed_v3.both_sexes.finemapping.pheno.tsv.gz
│   │   ├── I10.gwas.imputed_v3.both_sexes.finemapping.tsv.gz
│   │   ├── SNPs_only.log
│   │   ├── SNPs_only.recode.vcf
│   │   ├── Untitled.ipynb
│   │   ├── dgwas_subset_flt.pqt
│   │   ├── dgwas_subset_flt.tsv
│   │   ├── fgwas.llk
│   │   ├── fgwas.params
│   │   ├── file
│   │   ├── funiq.txt
│   │   ├── out.log
│   │   ├── out.recode.vcf
│   │   ├── split_by_chr.sh
│   │   ├── test.bfs.gz
│   │   ├── test.bfs.txt
│   │   ├── test.llk
│   │   ├── test.params
│   │   ├── test.ridgeparams
│   │   ├── test.segbfs.gz
│   │   ├── variants.vcf
│   │   ├── variants_no_indels.vcf
│   │   ├── variants_no_indels.vcf~
│   │   ├── vars_by_chr
│   │   │   ├── variants_no_indels.chr1.vcf
│   │   │   ├── variants_no_indels.chr10.vcf
│   │   │   ├── variants_no_indels.chr11.vcf
│   │   │   ├── variants_no_indels.chr12.vcf
│   │   │  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob

In [3]:
from kipoi_gwas.regulatory_features import *

In [10]:
chromosome='12'
tissue_type='Aorta'
dfeatsp=f'{ddir}/ensembl/processed/dgff_regulation_combo.chr{chromosome}.{tissue_type}.pqt'

In [11]:
dfeats=pd.read_parquet(dfeatsp,engine='fastparquet')

In [12]:
dfeats.head()

,chromosome,source,type,start,end,score,strand,phase,tissue type,regulatory_feature_stable_id
index,,,,,,,,,,
25,12,Regulatory_Build,CTCF_binding_site,49681801,49683000,.,.,.,Aorta,ENSR00000069585
36,12,Regulatory_Build,promoter_flanking_region,6887201,6888800,.,.,.,Aorta,ENSR00000065041
46,12,Regulatory_Build,promoter,109592200,109593401,.,.,.,Aorta,ENSR00000076948
62,12,Regulatory_Build,TF_binding_site,71946831,71947398,.,.,.,Aorta,ENSR00000072663
66,12,Regulatory_Build,open_chromatin_region,62397539,62397814,.,.,.,Aorta,ENSR00000360225


In [19]:
dvariantsp=f'{ddir}/workflow/output/I10.gwas.imputed_v3.both_sexes/gwas-table-unannotated.tsv.gz'
dvariants=pd.read_csv(dvariantsp, sep=' ')

In [21]:
dvariants=dvariants.set_index('SNPID')

In [ ]:
def if_feats(pos,feat,chromosome,dfeats):
    dfeats_=dfeats.loc[((dfeats['chromosome']==chromosome) & (dfeats['type']==feat)),:]
    dfeats_=dfeats_.reset_index()
    return any([True if ((pos>=dfeats_.loc[idx,'start']) and (pos<=dfeats_.loc[idx,'end'])) else False for idx in dfeats_.index])
feats=dfeats['type'].unique()
for feat in feats:
    dvariants.loc[:,feat]=dvariants.apply(lambda x : if_feats(x['POS'],feat,chromosome,dfeats),axis=1) 

In [22]:
for vid in dvariants.index:
    pos=dvariants.loc[vid,'POS']
    feats=dfeats['type'].unique()
    for feat in feats:
        for fid in dfeats.index:
            dvariants.loc[vid,feat]=True if ((pos => dfeats.loc[((dfeats['chromosome']==chromosome) & (dfeats['type']==feat)),'start']) and (pos => dfeats.loc[((dfeats['chromosome']==chromosome) & (dfeats['type']==feat)),'start'])) else False
            break
        break

SyntaxError: unexpected EOF while parsing (<ipython-input-22-8b4447f1fab1>, line 1)

In [20]:
dvariants.head()

,SNPID,CHR,POS,Z,F,SEGNUMBER,pval,NCONTROL,NCASE
0,3:149461748:T:C,3,149461748,-0.225786,0.060348,1,0.821368,360407,787
1,3:149462893:T:G,3,149462893,0.674998,0.488693,1,0.499678,360407,787
2,3:149463939:T:C,3,149463939,-0.717405,0.017110,1,0.473125,360407,787
3,3:149464402:CA:C,3,149464402,1.147810,0.494635,1,0.251046,360407,787
4,3:149464427:A:G,3,149464427,0.785214,0.410530,1,0.432329,360407,787


In [23]:
dfeats.head()

,chromosome,source,type,start,end,score,strand,phase,tissue type,regulatory_feature_stable_id
index,,,,,,,,,,
25,12,Regulatory_Build,CTCF_binding_site,49681801,49683000,.,.,.,Aorta,ENSR00000069585
36,12,Regulatory_Build,promoter_flanking_region,6887201,6888800,.,.,.,Aorta,ENSR00000065041
46,12,Regulatory_Build,promoter,109592200,109593401,.,.,.,Aorta,ENSR00000076948
62,12,Regulatory_Build,TF_binding_site,71946831,71947398,.,.,.,Aorta,ENSR00000072663
66,12,Regulatory_Build,open_chromatin_region,62397539,62397814,.,.,.,Aorta,ENSR00000360225
